In [1]:
# Import modules
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from selenium import webdriver
import pandas as pd
import time
from secret import username, password
from sqlalchemy import create_engine

In [2]:
# URL of page to be scraped
url = 'https://en.wikipedia.org/wiki/List_of_highest-grossing_films'

### Mac (Irina) Connection

In [3]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [4]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### Windows (Clay) Connection

In [5]:
# executable_path = {'executable_path': 'C:/Users/Clay/chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)

In [6]:
browser.visit(url)

In [7]:
tables = pd.read_html(url)

In [8]:
table_df = tables[0]

In [9]:
table_df.set_index('Rank')
table_df.head()

,Rank,Peak,Title,Worldwide gross,Year,Reference(s)
0,1,1,Avengers: Endgame,"$2,797,800,564",2019,[# 1][# 2]
1,2,1,Avatar,"$2,790,439,000",2009,[# 3][# 4]
2,3,1,Titanic,"$2,194,439,542",1997,[# 5][# 6]
3,4,3,Star Wars: The Force Awakens,"$2,068,223,624",2015,[# 7][# 8]
4,5,4,Avengers: Infinity War,"$2,048,359,754",2018,[# 9][# 10]


In [10]:
table_df.drop(columns=['Peak', 'Year', 'Reference(s)'], inplace=True)

In [11]:
table_df = table_df.reset_index()

In [12]:
ww_gross_series = table_df['Worldwide gross']

In [13]:
len(ww_gross_series)

50

In [14]:
type(ww_gross_series)

pandas.core.series.Series

In [15]:
list = []
for i in ww_gross_series:
    print(i.split('$')[-1])
    list.append(i.split('$')[-1]) 

2,797,800,564
2,790,439,000
2,194,439,542
2,068,223,624
2,048,359,754
1,671,713,208
1,656,943,394
1,518,812,988
1,516,045,911
1,450,026,933
1,402,805,868
1,346,913,161
1,341,932,398
1,332,539,889
1,309,484,461
1,290,000,000
1,263,521,126
1,242,805,359
1,238,764,765
1,214,811,252
1,159,398,397
1,153,304,495
1,148,161,807
1,142,219,401
1,131,927,996
1,128,274,794
1,123,794,079
1,108,561,013
1,104,054,072
1,084,939,099
1,074,251,311
1,074,144,248
1,073,394,593
1,066,969,703
1,066,179,725
1,056,057,273
1,050,693,953
1,045,713,802
1,034,799,409
1,029,939,903
1,028,570,889
1,027,044,677
1,025,467,110
1,023,784,195
1,021,103,568
1,004,934,033
978,087,613
976,920,103
970,761,885
968,483,777


In [16]:
split_list = []
for i in list:
    i = i.split(',')
    split_list.append(''.join(i)) 

In [17]:
merged_list = []
for i in split_list:
    print(''.join(i))
    merged_list.append(''.join(i)) 

2797800564
2790439000
2194439542
2068223624
2048359754
1671713208
1656943394
1518812988
1516045911
1450026933
1402805868
1346913161
1341932398
1332539889
1309484461
1290000000
1263521126
1242805359
1238764765
1214811252
1159398397
1153304495
1148161807
1142219401
1131927996
1128274794
1123794079
1108561013
1104054072
1084939099
1074251311
1074144248
1073394593
1066969703
1066179725
1056057273
1050693953
1045713802
1034799409
1029939903
1028570889
1027044677
1025467110
1023784195
1021103568
1004934033
978087613
976920103
970761885
968483777


In [18]:
len(merged_list)

50

In [19]:
merged_list

['2797800564',
 '2790439000',
 '2194439542',
 '2068223624',
 '2048359754',
 '1671713208',
 '1656943394',
 '1518812988',
 '1516045911',
 '1450026933',
 '1402805868',
 '1346913161',
 '1341932398',
 '1332539889',
 '1309484461',
 '1290000000',
 '1263521126',
 '1242805359',
 '1238764765',
 '1214811252',
 '1159398397',
 '1153304495',
 '1148161807',
 '1142219401',
 '1131927996',
 '1128274794',
 '1123794079',
 '1108561013',
 '1104054072',
 '1084939099',
 '1074251311',
 '1074144248',
 '1073394593',
 '1066969703',
 '1066179725',
 '1056057273',
 '1050693953',
 '1045713802',
 '1034799409',
 '1029939903',
 '1028570889',
 '1027044677',
 '1025467110',
 '1023784195',
 '1021103568',
 '1004934033',
 '978087613',
 '976920103',
 '970761885',
 '968483777']

In [20]:
# Rename the columns to be able to merge the data 
table_df.rename(columns={"Title": "name", "Rank": "rank", "Worldwide gross":"ww_gross"}, inplace=True)

In [21]:
# Change revenue column from string to big integer
table_df['ww_gross'] = merged_list

In [22]:
table_df.head()

,index,rank,name,ww_gross
0,0,1,Avengers: Endgame,2797800564
1,1,2,Avatar,2790439000
2,2,3,Titanic,2194439542
3,3,4,Star Wars: The Force Awakens,2068223624
4,4,5,Avengers: Infinity War,2048359754


### Connect to local database

In [23]:
connection_string = f'{username}:{password}@localhost:5432/movies_db'
engine = create_engine(f'postgresql://{connection_string}')

In [24]:
engine.table_names()

['combined_revenue',
 'international_revenue',
 'best_200_directors',
 'domestic_revenue']

### Use pandas to load csv converted DataFrame into database

In [25]:
table_df.to_sql(name='ww_top_50_movies', con=engine, if_exists='append', index=False)

### Confirm data has been added by querying the international table

In [26]:
pd.read_sql_query('select * from ww_top_50_movies', con=engine).head()

,index,rank,name,ww_gross
0,0,1,Avengers: Endgame,2797800564
1,1,2,Avatar,2790439000
2,2,3,Titanic,2194439542
3,3,4,Star Wars: The Force Awakens,2068223624
4,4,5,Avengers: Infinity War,2048359754


### Web Scrape Box Office Mojo

In [29]:
mojo_url = 'https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW'

In [30]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [31]:
browser.visit(mojo_url)

In [32]:
tables = pd.read_html(mojo_url)

In [33]:
numbers_table_df = tables[0]

In [34]:
numbers_table_df

,Rank,Title,Lifetime Gross,Year
0,1,Avengers: Endgame,"$2,797,800,564",2019
1,2,Avatar,"$2,790,439,000",2009
2,3,Titanic,"$2,195,169,138",1997
3,4,Star Wars: Episode VII - The Force Awakens,"$2,068,223,624",2015
4,5,Avengers: Infinity War,"$2,048,359,754",2018
...,...,...,...,...
195,196,The Revenant,"$532,950,503",2015
196,197,The Meg,"$530,254,422",2016
197,198,Ralph Breaks the Internet,"$529,323,962",2018
198,199,Hotel Transylvania 3: Summer Vacation,"$528,583,774",2018


In [35]:
mojo_url_2 = 'https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW&offset=200'

In [30]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [31]:
browser.visit(mojo_url)

In [32]:
tables = pd.read_html(mojo_url)

In [33]:
numbers_table_df = tables[0]

In [34]:
numbers_table_df

,Rank,Title,Lifetime Gross,Year
0,1,Avengers: Endgame,"$2,797,800,564",2019
1,2,Avatar,"$2,790,439,000",2009
2,3,Titanic,"$2,195,169,138",1997
3,4,Star Wars: Episode VII - The Force Awakens,"$2,068,223,624",2015
4,5,Avengers: Infinity War,"$2,048,359,754",2018
...,...,...,...,...
195,196,The Revenant,"$532,950,503",2015
196,197,The Meg,"$530,254,422",2016
197,198,Ralph Breaks the Internet,"$529,323,962",2018
198,199,Hotel Transylvania 3: Summer Vacation,"$528,583,774",2018


In [3]:
mojo_list = []
base_url = 'https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW'
page_num = 0
table_mojo_df = []
table_on_page = ""

for i in range(0, 10200, 200):
    if page_num < 100:
        print("Printing first 200 rows.")
        url=(f"{base_url}")
        print(url)
        executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
        browser = Browser('chrome', **executable_path, headless=False)
        browser.visit(url)
        time.sleep(5)
        print("sleep 5")
        tables = pd.read_html(url)
        time.sleep(5)
        print("sleep 5")
        table_mojo_df = tables[0]
        browser.quit()
        page_num += 200
        table_mojo_df = table_mojo_df.append(table_mojo_df)
        table_page = (f"Table_{str(i)}")
        print(table_page)
        print(type(table_page))
        table_page = table_mojo_df
        print(table_page)
        table_mojo_df.to_csv(f"csv/Table_{str(i)}.csv")
        print('-------------------------------------')
    else:
        print(f"Printing pages {i+1} to {i+200}")
        url=(f"{base_url}&offset={str(i)}")
        print(url)
        executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
        browser = Browser('chrome', **executable_path, headless=False)
        browser.visit(url)
        time.sleep(5)
        print("sleep 5")
        tables = pd.read_html(url)
        time.sleep(5)
        print("sleep 5")
        table_mojo_df = tables[0]
        browser.quit()
        page_num += 200
        table_mojo_df = table_mojo_df.append(table_mojo_df)
#         table_page = str(i)
        table_page = (f"Table_{str(i)}")
        print(table_page)
        print(type(table_page))
        table_page = table_mojo_df
        print(table_page)
        table_mojo_df.to_csv(f"csv/Table_{str(i)}.csv")
        print('-------------------------------------')

    

Printing first 200 rows.
https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW
sleep 5
sleep 5
Table_0
<class 'str'>
     Rank                                       Title  Lifetime Gross  Year
0       1                           Avengers: Endgame  $2,797,800,564  2019
1       2                                      Avatar  $2,790,439,000  2009
2       3                                     Titanic  $2,195,169,138  1997
3       4  Star Wars: Episode VII - The Force Awakens  $2,068,223,624  2015
4       5                      Avengers: Infinity War  $2,048,359,754  2018
..    ...                                         ...             ...   ...
195   196                                The Revenant    $532,950,503  2015
196   197                                     The Meg    $530,254,422  2016
197   198                   Ralph Breaks the Internet    $529,323,962  2018
198   199       Hotel Transylvania 3: Summer Vacation    $528,583,774  2018
199   200                           

sleep 5
sleep 5
Table_1800
<class 'str'>
     Rank                Title Lifetime Gross  Year
0    1801  Season of the Witch    $91,627,228  2011
1    1802                Crawl    $91,542,097  2019
2    1803      Five Feet Apart    $91,522,094  2019
3    1804      Friday the 13th    $91,509,154  2009
4    1805    Rambo: Last Blood    $91,490,353  2019
..    ...                  ...            ...   ...
195  1996    Go Away Mr. Tumor    $80,606,893  2015
196  1997        Vampires Suck    $80,547,866  2010
197  1998       Den of Thieves    $80,509,622  2018
198  1999            Halloween    $80,460,948  2007
199  2000  Muppets Most Wanted    $80,383,113  2014

[400 rows x 4 columns]
-------------------------------------
Printing pages 2001 to 2200
https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW&offset=2000
sleep 5
sleep 5
Table_2000
<class 'str'>
     Rank                                     Title Lifetime Gross  Year
0    2001                                    Takers   

sleep 5
sleep 5
Table_3600
<class 'str'>
     Rank                                   Title Lifetime Gross  Year
0    3601    The Adventures of Rocky & Bullwinkle    $35,134,820  2000
1    3602                             Miss Potter    $35,078,241  2006
2    3603                                   Alfie    $35,060,882  2004
3    3604                    My Week with Marilyn    $35,057,696  2011
4    3605                          The Liquidator    $35,046,130  2017
..    ...                                     ...            ...   ...
195  3796  The House That Never Dies: Reawakening    $32,433,588  2017
196  3797                             Black Sheep    $32,417,995  1996
197  3798                  The Invention of Lying    $32,406,507  2009
198  3799                                 Dhoom 2    $32,396,427  2006
199  3800                     Rumble in the Bronx    $32,392,047  1996

[400 rows x 4 columns]
-------------------------------------
Printing pages 3801 to 4000
https://www.boxof

sleep 5
sleep 5
Table_5400
<class 'str'>
     Rank                         Title Lifetime Gross  Year
0    5401                From Me to You    $18,033,727  2010
1    5402  Mr. Pride vs. Miss Prejudice    $18,019,651  2017
2    5403                    Cape No. 7    $18,015,600  2008
3    5404             The Country Bears    $18,012,097  2002
4    5405         Harlock: Space Pirate    $18,010,317  2013
..    ...                           ...            ...   ...
195  5596      Kung Pow: Enter the Fist    $16,994,625  2002
196  5597                Death Sentence    $16,974,459  2007
197  5598                Chinese Puzzle    $16,968,297  2013
198  5599          Till Luck Do Us Part    $16,963,608  2012
199  5600                Catch That Kid    $16,951,702  2004

[400 rows x 4 columns]
-------------------------------------
Printing pages 5601 to 5800
https://www.boxofficemojo.com/chart/top_lifetime_gross/?area=XWW&offset=5600
sleep 5
sleep 5
Table_5600
<class 'str'>
     Rank          

sleep 5
sleep 5
Table_7200
<class 'str'>
     Rank                                           Title Lifetime Gross  Year
0    7201                                  Thread of Lies    $10,522,797  2014
1    7202                                         Get Low    $10,522,511  2010
2    7203                            Black Widow Business    $10,504,345  2016
3    7204  Até Que a Sorte nos Separe 3: A Falência Final    $10,502,937  2015
4    7205                                        Anatomie    $10,501,958  2000
..    ...                                             ...            ...   ...
195  7396                                    The Deep End    $10,031,529  2001
196  7397                             Tear This Heart Out    $10,031,226  2008
197  7398                Kisarazu Cat's Eye: Nihon Series    $10,029,866  2003
198  7399                                    Agathe Cléry    $10,025,910  2008
199  7400                                     Crying Fist    $10,024,751  2005

[400 rows 

sleep 5
sleep 5
Table_8800
<class 'str'>
     Rank                                              Title Lifetime Gross  \
0    8801                                               Goon     $6,985,158   
1    8802                                      Psychokinesis     $6,981,970   
2    8803       Children of the Corn II: The Final Sacrifice     $6,980,986   
3    8804                                       Donnie Darko     $6,979,093   
4    8805                                Kôdaike no hitobito     $6,978,689   
..    ...                                                ...            ...   
195  8996                                   Forced Vengeance     $6,660,333   
196  8997                             Where the Buffalo Roam     $6,659,377   
197  8998                                        Son of Saul     $6,659,121   
198  8999  InuYasha the Movie 3: Swords of an Honorable R...     $6,654,841   
199  9000                                        April Fools     $6,652,777   

     Year 

In [47]:
type(table_page)

pandas.core.frame.DataFrame

In [53]:
table_page.tail()

,Rank,Title,Lifetime Gross,Year
195,796,JFK,"$205,405,498",1991
196,797,White House Down,"$205,366,737",2013
197,798,Fun with Dick and Jane,"$204,681,899",2005
198,799,Eat Pray Love,"$204,594,016",2010
199,800,You Don't Mess with the Zohan,"$204,313,400",2008


In [53]:
table_page.tail()

,Rank,Title,Lifetime Gross,Year
195,796,JFK,"$205,405,498",1991
196,797,White House Down,"$205,366,737",2013
197,798,Fun with Dick and Jane,"$204,681,899",2005
198,799,Eat Pray Love,"$204,594,016",2010
199,800,You Don't Mess with the Zohan,"$204,313,400",2008


In [54]:
table_mojo_df.head()

,Rank,Title,Lifetime Gross,Year
0,601,Us,"$255,184,580",2019
1,602,Cliffhanger,"$255,000,211",1993
2,603,Baahubali 2: The Conclusion,"$254,158,390",2017
3,604,Babe,"$254,134,910",1995
4,605,Men in Black: International,"$253,890,701",2019


In [55]:
table_mojo_df.tail()

,Rank,Title,Lifetime Gross,Year
195,796,JFK,"$205,405,498",1991
196,797,White House Down,"$205,366,737",2013
197,798,Fun with Dick and Jane,"$204,681,899",2005
198,799,Eat Pray Love,"$204,594,016",2010
199,800,You Don't Mess with the Zohan,"$204,313,400",2008


In [58]:
test = str(400)

## Prepare international revenue data set for loading to SQL

In [30]:
# Read the clean, combined csv file
master_df = pd.read_csv("Master_Table.csv")

In [29]:
# Remove duplicated indexes
clean_df = master_df.drop(index=master_df.index[[6643, 8994]])

In [32]:
# Getting the duplicate movie by title and year released. 
duplicated_master_df = master_df.loc[master_df.duplicated(subset=["Title","Year"]), "Title"].unique()
duplicated_master_df

array(['War of the Buttons', 'My Love'], dtype=object)

In [33]:
# Get all the data for the duplicate movie 
duplicated_master_df = master_df.loc[master_df["Title"] == "War of the Buttons"]
duplicated_master_df

,Rank,Title,Year,Lifetime Gross,GrossNumeric
5983,5984,War of the Buttons,2011,"$15,082,409",15082409
6643,6644,War of the Buttons,2011,"$12,326,773",12326773


In [34]:
# Get all the data for the duplicate movie 
duplicated_master_df = master_df.loc[master_df["Title"] == "My Love"]
duplicated_master_df

,Rank,Title,Year,Lifetime Gross,GrossNumeric
5334,5335,My Love,2007,"$18,525,631",18525631
8994,8995,My Love,2007,"$6,660,413",6660413


In [35]:
# reset index
master_df.reset_index()

,index,Rank,Title,Year,Lifetime Gross,GrossNumeric
0,0,1,Avengers: Endgame,2019,"$2,797,800,564",2797800564
1,1,2,Avatar,2009,"$2,790,439,000",2790439000
2,2,3,Titanic,1997,"$2,195,169,138",2195169138
3,3,4,Star Wars: Episode VII - The Force Awakens,2015,"$2,068,223,624",2068223624
4,4,5,Avengers: Infinity War,2018,"$2,048,359,754",2048359754
...,...,...,...,...,...,...
10195,10195,10196,Swiss Army Man,2016,"$4,935,501",4935501
10196,10196,10197,Mes héros,2012,"$4,934,927",4934927
10197,10197,10198,Nymphomaniac: Vol. II,2014,"$4,934,725",4934725
10198,10198,10199,The Hairdresser,2010,"$4,934,394",4934394


In [36]:
# Drop row by index
clean_df = master_df.drop(index=master_df.index[[6643, 8994]])

In [37]:
len(clean_df)

10198

In [38]:
# Select columns
clean_df = clean_df[['Rank', 'Title', 'Year', 'GrossNumeric']]

In [39]:
# Rename columns
international_v2 = clean_df.rename(columns={'Rank': 'rank', 'Title': 'name_international', 
                'Year': 'year', 'GrossNumeric': 'international_revenue_usd'})

In [40]:
international_v2.head()

,rank,name_international,year,international_revenue_usd
0,1,Avengers: Endgame,2019,2797800564
1,2,Avatar,2009,2790439000
2,3,Titanic,1997,2195169138
3,4,Star Wars: Episode VII - The Force Awakens,2015,2068223624
4,5,Avengers: Infinity War,2018,2048359754


In [41]:
international_v2.set_index('rank', inplace=True)

In [42]:
international_v2.head()

,name_international,year,international_revenue_usd
rank,,,
1,Avengers: Endgame,2019,2797800564
2,Avatar,2009,2790439000
3,Titanic,1997,2195169138
4,Star Wars: Episode VII - The Force Awakens,2015,2068223624
5,Avengers: Infinity War,2018,2048359754


In [43]:
# Change name column to lower case to easier join 
international_v2['name_international'] = international_v2['name_international'].str.capitalize()

In [44]:
international_v2.head()

,name_international,year,international_revenue_usd
rank,,,
1,Avengers: endgame,2019,2797800564
2,Avatar,2009,2790439000
3,Titanic,1997,2195169138
4,Star wars: episode vii - the force awakens,2015,2068223624
5,Avengers: infinity war,2018,2048359754


In [45]:
international_v2.to_csv('Resources/Master_table_v3.csv')

### Clean domestic revenue data¶

In [46]:
# Read csv file into df
domestic_df = pd.read_csv('Resources/datasets_movies_v3.csv')

In [48]:
# Make names capitalized
domestic_df['name'] = domestic_df['name'].str.capitalize()

In [49]:
# Drop year and country columns
domestic_df = domestic_df.drop(columns=['year','country'])

In [50]:
domestic_df.head()

,budget,company,director,genre,gross,name,rating,released,runtime,score,star,votes,writer
0,8000000,Columbia Pictures Corporation,Rob Reiner,Adventure,52287414,Stand by me,R,8/22/86,89,8.1,Wil Wheaton,299174,Stephen King
1,6000000,Paramount Pictures,John Hughes,Comedy,70136369,Ferris bueller's day off,PG-13,6/11/86,103,7.8,Matthew Broderick,264740,John Hughes
2,15000000,Paramount Pictures,Tony Scott,Action,179800601,Top gun,PG,5/16/86,110,6.9,Tom Cruise,236909,Jim Cash
3,18500000,Twentieth Century Fox Film Corporation,James Cameron,Action,85160248,Aliens,R,7/18/86,137,8.4,Sigourney Weaver,540152,James Cameron
4,9000000,Walt Disney Pictures,Randal Kleiser,Adventure,18564613,Flight of the navigator,PG,8/1/86,90,6.9,Joey Cramer,36636,Mark H. Baker


In [51]:
# Getting the duplicate movie by title and year released. 
duplicated_domestic_df = domestic_df.loc[domestic_df.duplicated(subset=["name"]), "name"].unique()
duplicated_domestic_df

array(['Dream lover', 'Boiling point', "Nobody's fool", 'Heat',
       'It takes two', 'Rumpelstiltskin', 'Hamlet', 'Bulletproof',
       'The cure', 'Gladiator', 'Chocolat', 'Where the heart is',
       'Saving grace', 'The in crowd', 'Behind enemy lines', 'Out cold',
       'Pulse', 'Insomnia', 'Hero', 'Bad company', 'Crossroads',
       'The rookie', 'The hunted', 'It runs in the family',
       'The phantom of the opera', 'Man on fire', 'Taxi', 'Proof',
       'The proposition', 'Fever pitch', 'Man of the house', 'Invincible',
       'The guardian', 'Running scared', 'Fearless', 'Black sheep',
       'Hairspray', 'Sunshine', 'The hitcher', 'Flawless', 'Twilight',
       'The eye', 'Mother', 'The stepfather', 'The unborn',
       "Everybody's fine", 'Trust', 'Fair game', 'Death at a funeral',
       'Morning glory', 'The girl with the dragon tattoo', 'Priest',
       'The three musketeers', 'Jane eyre', 'Unknown', 'Trespass',
       'The avengers', 'Les misèrables', 'Anna karenina',

In [52]:
# Check duplicated movie
duplicated_domestic_df = domestic_df.loc[domestic_df["name"] == "Dream lover"]
duplicated_domestic_df

,budget,company,director,genre,gross,name,rating,released,runtime,score,star,votes,writer
192,0,Metro-Goldwyn-Mayer (MGM),Alan J. Pakula,Crime,502237,Dream lover,R,4/2/86,104,5.1,Kristy McNichol,405,Jon Boorstin
1652,0,PolyGram Filmed Entertainment,Nicholas Kazan,Drama,256264,Dream lover,R,5/6/94,103,6.2,James Spader,3327,Nicholas Kazan


In [55]:
# Drop duplicates and keep first
domestic_revenue_v2 = domestic_df.drop_duplicates(subset=['name'], keep='first')
len(domestic_revenue_v2)

6731

In [58]:
domestic_revenue_v2 = domestic_revenue_v2.rename(columns={'gross': 'domestic_revenue_usd',
                                        'name': 'name_domestic'})

In [60]:
domestic_revenue_v2.head(2)

,budget,company,director,genre,domestic_revenue_usd,name_domestic,rating,released,runtime,score,star,votes,writer
0,8000000,Columbia Pictures Corporation,Rob Reiner,Adventure,52287414,Stand by me,R,8/22/86,89,8.1,Wil Wheaton,299174,Stephen King
1,6000000,Paramount Pictures,John Hughes,Comedy,70136369,Ferris bueller's day off,PG-13,6/11/86,103,7.8,Matthew Broderick,264740,John Hughes


In [61]:
# Write to csv file
domestic_revenue_v2.to_csv('Resources/Domestic_v3.csv')

### Use pandas to push data into database

In [62]:
# Connect to the local database
connection_string = f'{username}:{password}@localhost:5432/movies_db'
engine = create_engine(f'postgresql://{connection_string}')

In [63]:
# Check existing tables
engine.table_names()

['combined_revenue',
 'ww_top_50_movies',
 'international_revenue',
 'best_200_directors',
 'domestic_revenue']

In [64]:
# Create international revenue table in the database
international_v2.to_sql(name='international_revenue', con=engine, if_exists='append', index=False)

In [65]:
# Create domesic revenue table in the database
domestic_revenue_v2.to_sql(name='domestic_revenue', con=engine, if_exists='append', index=False)

In [66]:
engine.table_names()

['combined_revenue',
 'ww_top_50_movies',
 'international_revenue',
 'best_200_directors',
 'domestic_revenue']

### Confirm data has been added by querying the international table

In [69]:
pd.read_sql_query("select * from international_revenue", con=engine).head(2)

,name_international,year,international_revenue_usd
0,Avengers: endgame,2019,2797800564
1,Avatar,2009,2790439000


In [70]:
pd.read_sql_query('select * from domestic_revenue', con=engine).head(2)

,budget,company,director,genre,domestic_revenue_usd,name_domestic,rating,released,runtime,score,star,votes,writer
0,8000000,Columbia Pictures Corporation,Rob Reiner,Adventure,52287414,Stand by me,R,8/22/86,89,8.1,Wil Wheaton,299174,Stephen King
1,6000000,Paramount Pictures,John Hughes,Comedy,70136369,Ferris bueller's day off,PG-13,6/11/86,103,7.8,Matthew Broderick,264740,John Hughes


### Best movie directors of all times - web scraping

In [72]:
# Setup the urls for scraping
best_director_url  = 'https://www.imdb.com/list/ls056848274/'
best_director_url2 = 'https://www.imdb.com/list/ls056848274/?sort=list_order,asc&mode=detail&page=2'

In [73]:
# Setup the browser
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [74]:
# Visit the browser
browser.visit(best_director_url)

In [76]:
# get results by h3
results = browser.find_by_css("h3")

In [77]:
# Read the text from h3 elements in the results
list_original = []
for i in results:
    print(i.text)
    list_original.append(i.text)

1. Steven Spielberg
2. Martin Scorsese
3. Alfred Hitchcock
4. Stanley Kubrick
5. Quentin Tarantino
6. Orson Welles
7. Francis Ford Coppola
8. Ridley Scott
9. Akira Kurosawa
10. Joel Coen
11. Ethan Coen
12. John Ford
13. Sergio Leone
14. Woody Allen
15. Billy Wilder
16. Clint Eastwood
17. Frank Capra
18. Charles Chaplin
19. Christopher Nolan
20. Roman Polanski
21. Ingmar Bergman
22. David Lynch
23. Fritz Lang
24. David Fincher
25. Peter Jackson
26. Federico Fellini
27. Tim Burton
28. James Cameron
29. Sidney Lumet
30. Brian De Palma
31. François Truffaut
32. Terrence Malick
33. David Cronenberg
34. William Wyler
35. Robert Zemeckis
36. Ang Lee
37. Howard Hawks
38. George Lucas
39. Mel Brooks
40. Terry Gilliam
41. Milos Forman
42. Steven Soderbergh
43. John Huston
44. Werner Herzog
45. Oliver Stone
46. David Lean
47. Danny Boyle
48. Rob Reiner
49. Buster Keaton
50. Michael Curtiz
51. Sam Raimi
52. Michael Mann
53. Robert Altman
54. Sam Peckinpah
55. Peter Weir
56. Spike Lee
57. Jean Reno

In [78]:
# Clean up the list
directors_list_100 = list_original[:100]
directors_list_100

['1. Steven Spielberg',
 '2. Martin Scorsese',
 '3. Alfred Hitchcock',
 '4. Stanley Kubrick',
 '5. Quentin Tarantino',
 '6. Orson Welles',
 '7. Francis Ford Coppola',
 '8. Ridley Scott',
 '9. Akira Kurosawa',
 '10. Joel Coen',
 '11. Ethan Coen',
 '12. John Ford',
 '13. Sergio Leone',
 '14. Woody Allen',
 '15. Billy Wilder',
 '16. Clint Eastwood',
 '17. Frank Capra',
 '18. Charles Chaplin',
 '19. Christopher Nolan',
 '20. Roman Polanski',
 '21. Ingmar Bergman',
 '22. David Lynch',
 '23. Fritz Lang',
 '24. David Fincher',
 '25. Peter Jackson',
 '26. Federico Fellini',
 '27. Tim Burton',
 '28. James Cameron',
 '29. Sidney Lumet',
 '30. Brian De Palma',
 '31. François Truffaut',
 '32. Terrence Malick',
 '33. David Cronenberg',
 '34. William Wyler',
 '35. Robert Zemeckis',
 '36. Ang Lee',
 '37. Howard Hawks',
 '38. George Lucas',
 '39. Mel Brooks',
 '40. Terry Gilliam',
 '41. Milos Forman',
 '42. Steven Soderbergh',
 '43. John Huston',
 '44. Werner Herzog',
 '45. Oliver Stone',
 '46. David 

In [80]:
# Remove the rating from the string
clean_list1 = []
for i in directors_list_100:
    director = i.split('. ')[-1]
    print(director)
    clean_list1.append(director)

Steven Spielberg
Martin Scorsese
Alfred Hitchcock
Stanley Kubrick
Quentin Tarantino
Orson Welles
Francis Ford Coppola
Ridley Scott
Akira Kurosawa
Joel Coen
Ethan Coen
John Ford
Sergio Leone
Woody Allen
Billy Wilder
Clint Eastwood
Frank Capra
Charles Chaplin
Christopher Nolan
Roman Polanski
Ingmar Bergman
David Lynch
Fritz Lang
David Fincher
Peter Jackson
Federico Fellini
Tim Burton
James Cameron
Sidney Lumet
Brian De Palma
François Truffaut
Terrence Malick
David Cronenberg
William Wyler
Robert Zemeckis
Ang Lee
Howard Hawks
George Lucas
Mel Brooks
Terry Gilliam
Milos Forman
Steven Soderbergh
John Huston
Werner Herzog
Oliver Stone
David Lean
Danny Boyle
Rob Reiner
Buster Keaton
Michael Curtiz
Sam Raimi
Michael Mann
Robert Altman
Sam Peckinpah
Peter Weir
Spike Lee
Jean Renoir
Andrei Tarkovsky
Elia Kazan
John Cassavetes
Vincente Minnelli
Hayao Miyazaki
Frank Darabont
Kathryn Bigelow
George Cukor
Ron Howard
John Carpenter
Guy Ritchie
Tony Scott
Gus Van Sant
Preston Sturges
Eisenstein
Russel

In [81]:
# Open second page
browser.visit(best_director_url2)

In [82]:
results2 = browser.find_by_css("h3")

In [83]:
original_list2 = []
for i in results2:
#     print(i.text)
    original_list2.append(i.text)

In [84]:
directors_list2 = original_list2[:100]

In [85]:
clean_list2 = []
for i in directors_list2:
    director = i.split('. ')[-1]
    clean_list2.append(director)

In [86]:
clean_list2

['Jean-Luc Godard',
 'Tom Hooper',
 'William Friedkin',
 'Antoine Fuqua',
 'Sydney Pollack',
 'Bernardo Bertolucci',
 'John Lasseter',
 'David Ayer',
 'Abrams',
 'Mike Nichols',
 'Lars von Trier',
 'John Milius',
 'John McTiernan',
 'Costa-Gavras',
 'DeMille',
 'Darren Aronofsky',
 'Luc Besson',
 'Doug Liman',
 'Bryan Singer',
 'Richard Linklater',
 'Anthony Minghella',
 'Curtis Hanson',
 'Michael Haneke',
 'Griffith',
 'Alan Parker',
 'Edgar Wright',
 'Pedro Almodóvar',
 'Barry Levinson',
 'Michelangelo Antonioni',
 'Cameron Crowe',
 'Jim Jarmusch',
 'Vittorio De Sica',
 'Spike Jonze',
 'Harold Ramis',
 'Michael Moore',
 'Shane Meadows',
 'Ken Loach',
 'Bobby Farrelly',
 'Jacques Audiard',
 'Gore Verbinski',
 'Jay Roach',
 'Jim Sheridan',
 'John Landis',
 'Richard Donner',
 'Jason Reitman',
 'Robert Redford',
 'Andrew Dominik',
 'Joss Whedon',
 'George Miller',
 'David Zucker',
 'Romero',
 'Mike Leigh',
 'Russell',
 'Neil Jordan',
 'Wilfred Jackson',
 'Pakula',
 'George Roy Hill',
 'M

In [88]:
# Combined two lists together
directors = clean_list1 + clean_list2

In [89]:
len(directors)

200

In [90]:
directors_df = pd.DataFrame(directors)

In [91]:
len(directors_df)

200

In [92]:
# Rename the column
directors_df.rename(columns={0: 'directors'}, inplace=True)

In [93]:
directors_df.reset_index()

,index,directors
0,0,Steven Spielberg
1,1,Martin Scorsese
2,2,Alfred Hitchcock
3,3,Stanley Kubrick
4,4,Quentin Tarantino
...,...,...
195,195,Theodoros Angelopoulos
196,196,James Whale
197,197,Terence Fisher
198,198,Chuck Jones


In [94]:
# Save directors to csv
directors_df.to_csv('Resources/Best_directors_200.csv')

#### Push Directors df to the database

In [95]:
directors_df.to_sql(name='best_200_directors', con=engine, if_exists='append', index=False)

In [96]:
# Check if written
pd.read_sql_query('select * from best_200_directors', con=engine).head()

,directors
0,Steven Spielberg
1,Martin Scorsese
2,Alfred Hitchcock
3,Stanley Kubrick
4,Quentin Tarantino


In [97]:
engine.table_names()

['combined_revenue',
 'ww_top_50_movies',
 'international_revenue',
 'best_200_directors',
 'domestic_revenue']